In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy BiqQuery ML Model on Vertex AI Model Registry and make predictions

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_registry/bqml_vertexai_model_registry.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_registry/bqml_vertexai_model_registry.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_registry/bqml_vertexai_model_registry.ipynb" target="_blank">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This tutorial demonstrates how to train a model with BigQuery ML and upload it on Vertex AI Model Registry, then make batch predictions.

Learn more about [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction) and Learn more about [BigQuery ML](https://cloud.google.com/vertex-ai/docs/beginner/bqml).

### Objective

In this tutorial, you learn how to use `Vertex AI Model Registry` with `BigQuery ML` and make batch predictions:

This tutorial uses the following Google Cloud ML services and resources:

- `Vertex AI Model Registry`
- `Vertex AI Endpoint` resources
- `Vertex Explainable AI`
- `BigQuery ML`


The steps performed include:

- Train a model with `BigQuery ML`
- Upload the model to `Vertex AI Model Registry` 
- Create a `Vertex AI Endpoint` resource
- Deploy the `Model` resource to the `Endpoint` resource
- Make `prediction` requests to the model endpoint
- Run `batch prediction` job on the `Model` resource 


### Dataset

The dataset used for this tutorial is the Penguins dataset from <a href="https://cloud.google.com/bigquery/public-data" target="_blank">BigQuery public datasets</a>. This version of the dataset is used to predict the species of penguins from the available features like culmen-length, flipper-depth etc.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery ML

Learn about <a href="https://cloud.google.com/vertex-ai/pricing" target="_blank">Vertex AI
pricing</a> and <a href="https://cloud.google.com/bigquery/pricing" target="_blank">BigQuery pricing</a>, and use the <a href="https://cloud.google.com/products/calculator/" target="_blank">Pricing
Calculator</a>
to generate a cost estimate based on your projected usage.

### Install additional packages

Install the following packages required to execute this notebook. 

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-bigquery \
                         pyarrow --quiet

### Colab only: Uncomment the following cell to restart the kernel

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"
DATA_REGION = "US"

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Import libraries

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

### Initialize Vertex AI and BigQuery SDKs for Python

Initialize the Vertex AI and Big Query SDKs for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bqclient = bigquery.Client(project=PROJECT_ID)

## BigQuery ML introduction

BigQuery ML (BQML) provides the capability to train ML tabular models, such as classification and regression, in BigQuery using SQL syntax.

Learn more about <a href="https://cloud.google.com/bigquery-ml/docs" target="_blank">BigQuery ML documentation</a>.

### BigQuery table used for training

In [ ]:
# Define BigQuery table to be used for training

BQ_TABLE = "bigquery-public-data.ml_datasets.penguins"

### Create BigQuery dataset resource
First, you create an empty dataset resource in your project.

In [ ]:
BQ_DATASET_NAME = "penguins"
DATASET_QUERY = f"""CREATE SCHEMA {BQ_DATASET_NAME}"""

job = bqclient.query(DATASET_QUERY)
job.result()
print(job.state)

## Train BigQuery ML model and upload it to Vertex AI Model Registry
Next, you create and train a `BigQuery  ML` tabular regression model from the public dataset penguins and store the model in your project `Vertex AI Model Registry` using the `CREATE MODEL` statement. The model configuration is specified in the `OPTIONS` statement as follows:

- `model_type`: The type and archictecture of tabular model to train, e.g., LOGISTIC_REG.

- `labels`: The column which are the labels.

- `model_registry`: To register a BigQuery ML model to Vertex AI Model Registry, you must use `model_registry="vertex_ai"`.

During this stage, if you are using a supported model type, BQML exports metadata associated to the model to Vertex so it can run Vertex Explainable AI. Learn more about <a href="https://cloud.google.com/bigquery-ml/docs/vertex-xai" target="_blank">supported BQML model types</a>.

Learn more about the <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create" target="_blank">CREATE MODEL statement</a>.

Learn more about <a href="https://cloud.google.com/bigquery-ml/docs/managing-models-vertex" target="_blank">Managing BigQuery ML models in the Vertex AI Model Registry</a>.

In [ ]:
# Write the query to create Big Query ML model

MODEL_NAME = "penguins-lr"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='LOGISTIC_REG',
    labels = ['species'],
    model_registry='vertex_ai'
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

### Create BigQuery ML Model
Create the BigQuery ML model using the query above and the BigQuery client that you created previously:

In [ ]:
# Run the model creation query using BigQuery client

job = bqclient.query(MODEL_QUERY)
print(f"Job state: {job.state}\nJob Error:{job.errors}")

Check the job status:

In [ ]:
job.result()
print(job.state)

### Find the model in the Vertex Model Registry

You can use the `Vertex AI Model()` method with `model_name` parameter to find the automatically registered model.

In [ ]:
model = aiplatform.Model(model_name=MODEL_NAME)

print(model.gca_resource)

## Deploy Vertex AI Model resource to a Vertex AI Endpoint resource
You must deploy a model to an `endpoint` before that model can be used to serve online predictions; deploying a model associates physical resources with the model so it can serve online predictions with low latency. 

Learn more about <a href="https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#aiplatform_deploy_model_custom_trained_model_sample-python" target="_blank">Deploy a model using the Vertex AI API</a>


### Create a Vertex AI Endpoint resource

If you are deploying a model to an existing endpoint, you can skip this cell.

- `display_name`: Display name for the endpoint.
- `project`: The project ID on which you are creating an endpoint.
- `location`: The region where you are using Vertex AI.

In [ ]:
ENDPOINT_DISPLAY_NAME = "bqml-lr-model-endpoint"

endpoint = aiplatform.Endpoint.create(
    display_name=ENDPOINT_DISPLAY_NAME,
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint.display_name)
print(endpoint.resource_name)

### Deploy the Vertex AI Model resource to Vertex AI Endpoint resource

In [ ]:
DEPLOYED_NAME = "bqml-lr-penguins"

model.deploy(endpoint=endpoint, deployed_model_display_name=DEPLOYED_NAME)

print(model.display_name)
print(model.resource_name)

## Send prediction request to the Vertex AI Endpoint resource

Now that your Vertex AI Model resource is deployed to a Vertex AI `Endpoint` resource, you can do online predictions by sending prediction requests to the `Endpoint` resource.

Learn more about <a href="https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models" target="_blank">Get online predictions from custom-trained models</a>

In [ ]:
instance = {
    "island": "Dream",
    "culmen_length_mm": 36.6,
    "culmen_depth_mm": 18.4,
    "flipper_length_mm": 184.0,
    "body_mass_g": 3475.0,
    "sex": "FEMALE",
}

prediction = endpoint.predict([instance])
print(prediction)

## Send Explainable AI request to the Vertex AI Endpoint resource

If this is a supported BQML model type for Vertex Explainable AI, you can get explainations for your resource. To learn more about using Vertex AI Explainability on BQML models, see [Explainable AI for BigQuery ML models](https://cloud.google.com/bigquery-ml/docs/vertex-xai)


In [ ]:
instance = {
    "island": "Dream",
    "culmen_length_mm": 36.6,
    "culmen_depth_mm": 18.4,
    "flipper_length_mm": 184.0,
    "body_mass_g": 3475.0,
    "sex": "FEMALE",
}

explanation = endpoint.explain([instance])
print(explanation)

## Batch prediction on the BigQuery ML model
Here you request batch predictions directly from the BigQuery ML model; you don't need to deploy the model to an endpoint. For data types that support both batch and online predictions, use batch predictions when you don't require an immediate response and want to process accumulated data by using a single request.

Learn more abount <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict" target="_blank">The ML.PREDICT function</a>

In [ ]:
sql_ml_predict = f"""SELECT * FROM ML.PREDICT(MODEL `{PROJECT_ID}.{BQ_DATASET_NAME}.{MODEL_NAME}`, 
(SELECT
      *
    FROM
      `{BQ_TABLE}` LIMIT 10))"""

job = bqclient.query(sql_ml_predict)
prediction_result = job.result().to_arrow().to_pandas()

In [ ]:
display(prediction_result.head())

## Cleaning up

To clean up all Google Cloud resources used in this project, you can <a href="https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects" target="_blank">delete the Google Cloud
project</a> you used for the tutorial.

Learn more about <a href="https://cloud.google.com/bigquery-ml/docs/managing-models-vertex" target="_blank">Deleting BigQuery ML models from Vertex AI Model Registry</a>

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
# Delete the endpoint using the Vertex endpoint object
endpoint.undeploy_all()
endpoint.delete()

In [ ]:
# Delete BigQuery ML model

delete_query = f"""DROP MODEL `{PROJECT_ID}.{BQ_DATASET_NAME}.{MODEL_NAME}`"""
job = bqclient.query(delete_query)
job.result()

In [ ]:
# Delete the created BigQuery dataset
! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME